In [120]:
import os
import pandas as pd
import numpy as np
import altair as alt


npl = pd.read_csv(r'C:\Users\justi\Google Drive\_SMU\6390 Visualization\NPL_data.csv')
npl.head()
npl.columns=['time','NPLs','Loan Loss Reserves','Loan Loss Reserves Rate','NPL Rate']
npl.loc[9,'time'] = 'Dec 2019'
print(npl.dtypes)
print(npl)

time                       object
NPLs                       object
Loan Loss Reserves         object
Loan Loss Reserves Rate    object
NPL Rate                   object
dtype: object
       time    NPLs Loan Loss Reserves Loan Loss Reserves Rate NPL Rate
0      2014  $1.24              $0.89                    6.60%    4.30%
1      2015  $1.31              $0.82                    6.60%    3.70%
2      2016  $1.55              $1.21                    7.00%    5.00%
3      2017  $1.61              $1.27                    6.70%    4.80%
4      2018  $1.65              $1.50                    7.50%    5.90%
5   2019 Q1  $1.71              $1.66                    7.50%    6.20%
6   2019 Q2  $1.63              $1.51                    7.50%    6.20%
7   2019 Q3  $1.49              $1.61                    7.30%    5.60%
8   2019 Q4  $1.41              $1.44                    6.50%    5.80%
9  Dec 2019  $1.41              $1.43                    6.50%    5.20%


In [121]:
npl.NPLs.apply(type)

0    <class 'str'>
1    <class 'str'>
2    <class 'str'>
3    <class 'str'>
4    <class 'str'>
5    <class 'str'>
6    <class 'str'>
7    <class 'str'>
8    <class 'str'>
9    <class 'str'>
Name: NPLs, dtype: object

In [122]:
def clean(x):  #clean up the $'s
    if isinstance(x, str):
        return(x.replace('$','').replace('%',''))
    return(x)

for i in npl.columns[1:5]:
    
    npl[i] = npl[i].apply(clean).astype(float)

npl


,time,NPLs,Loan Loss Reserves,Loan Loss Reserves Rate,NPL Rate
0,2014,1.24,0.89,6.6,4.3
1,2015,1.31,0.82,6.6,3.7
2,2016,1.55,1.21,7.0,5.0
3,2017,1.61,1.27,6.7,4.8
4,2018,1.65,1.50,7.5,5.9
5,2019 Q1,1.71,1.66,7.5,6.2
6,2019 Q2,1.63,1.51,7.5,6.2
7,2019 Q3,1.49,1.61,7.3,5.6
8,2019 Q4,1.41,1.44,6.5,5.8
9,Dec 2019,1.41,1.43,6.5,5.2


In [123]:
npl.loc[10] = npl.loc[5:8].mean()
npl.iloc[10,0] = '2019'
npl.sort_values(by='time', inplace=True, ignore_index=True)
npl

,time,NPLs,Loan Loss Reserves,Loan Loss Reserves Rate,NPL Rate
0,2014,1.24,0.890,6.6,4.30
1,2015,1.31,0.820,6.6,3.70
2,2016,1.55,1.210,7.0,5.00
3,2017,1.61,1.270,6.7,4.80
4,2018,1.65,1.500,7.5,5.90
5,2019,1.56,1.555,7.2,5.95
6,2019 Q1,1.71,1.660,7.5,6.20
7,2019 Q2,1.63,1.510,7.5,6.20
8,2019 Q3,1.49,1.610,7.3,5.60
9,2019 Q4,1.41,1.440,6.5,5.80


In [192]:
npl_res = pd.melt(npl, id_vars='time', 
              value_vars=['NPLs','Loan Loss Reserves'],
              var_name='reserves', 
              value_name='billions')
npl_res.sort_values(by='time', inplace=True, ignore_index=True)
npl_res['group'] = 'y'
for i in range(len(npl_res)):
    if i<12:
        npl_res.loc[i,'group'] = 'y'
    elif i<20:
        npl_res.loc[i,'group'] = 'q'
    else:
        npl_res.loc[i,'group'] = 'm'
        

npl_perc = pd.melt(npl, id_vars='time',
             value_vars=['Loan Loss Reserves Rate', 'NPL Rate'],
             var_name='rates',
             value_name='percents')
npl_perc.sort_values(by='time', inplace=True, ignore_index=True)
npl_perc.percents = npl_perc.percents/100
npl_perc['group'] = 'y'
for i in range(len(npl_res)):
    if i<12:
        npl_perc.loc[i,'group'] = 'y'
    elif i<20:
        npl_perc.loc[i,'group'] = 'q'
    else:
        npl_perc.loc[i,'group'] = 'm'

npl_res,npl_perc

(        time            reserves  billions group
 0       2014                NPLs     1.240     y
 1       2014  Loan Loss Reserves     0.890     y
 2       2015                NPLs     1.310     y
 3       2015  Loan Loss Reserves     0.820     y
 4       2016                NPLs     1.550     y
 5       2016  Loan Loss Reserves     1.210     y
 6       2017                NPLs     1.610     y
 7       2017  Loan Loss Reserves     1.270     y
 8       2018                NPLs     1.650     y
 9       2018  Loan Loss Reserves     1.500     y
 10      2019                NPLs     1.560     y
 11      2019  Loan Loss Reserves     1.555     y
 12   2019 Q1                NPLs     1.710     q
 13   2019 Q1  Loan Loss Reserves     1.660     q
 14   2019 Q2                NPLs     1.630     q
 15   2019 Q2  Loan Loss Reserves     1.510     q
 16   2019 Q3  Loan Loss Reserves     1.610     q
 17   2019 Q3                NPLs     1.490     q
 18   2019 Q4  Loan Loss Reserves     1.440     q


In [193]:
npl_res['group'] = '0'
npl_res

,time,reserves,billions,group
0,2014,NPLs,1.240,0
1,2014,Loan Loss Reserves,0.890,0
2,2015,NPLs,1.310,0
3,2015,Loan Loss Reserves,0.820,0
4,2016,NPLs,1.550,0
5,2016,Loan Loss Reserves,1.210,0
6,2017,NPLs,1.610,0
7,2017,Loan Loss Reserves,1.270,0
8,2018,NPLs,1.650,0
9,2018,Loan Loss Reserves,1.500,0


In [194]:
list(npl_res), list(npl_perc)

(['time', 'reserves', 'billions', 'group'],
 ['time', 'rates', 'percents', 'group'])

In [195]:
# build the graph

years = npl_res.loc[0:11] # fiscal years
quarters = npl_res.loc[12:19] #2019 by quarter
month = npl_res.loc[20:21] #dec 2019

line_year = alt.Chart(years).mark_line(point=True).encode(
    alt.X('time:O',
          axis=alt.Axis(title='Fiscal Years')),
    alt.Y('billions', 
          axis=alt.Axis(format='$,.1f', title='Billions $'),
         scale=alt.Scale(domain=(.5,2.0))),
    color='reserves',
    strokeDash='reserves'
).properties(height=200, width=300,
            title='by Fiscal Year')
    
line_quarter = alt.Chart(quarters).mark_line(point=True).encode(
    alt.X('time:O',
          axis=alt.Axis(title='Quarters')),
    alt.Y('billions', 
          axis=alt.Axis(title='', format='$,.1f'),
         scale=alt.Scale(domain=(.5,2.0))),
    color='reserves',
    strokeDash='reserves'
).properties(height=200, width=200,
            title='in 2019 by Quarter')

line_month = alt.Chart(month).mark_bar().encode(
    alt.X('reserves',
          axis=alt.Axis(title='Reserve Type')),
    alt.Y('billions', 
          axis=alt.Axis(title='',format='$,.1f'),
          scale=alt.Scale(domain=(.5,2.0))),
    color='reserves',
    strokeDash='reserves'
).properties(height=200, width=200, title='in December 2019')
    
reserves = alt.hconcat(line_year, line_quarter, line_month).properties(title='Reserves')


In [196]:
yrs = npl_perc.loc[0:11] # fiscal years
qtrs = npl_perc.loc[12:19] #2019 by quarter
mons = npl_perc.loc[20:21] #dec 2019

line_yr = alt.Chart(yrs).mark_line(point=True).encode(
    alt.X('time:O',
          axis=alt.Axis(title='Fiscal Years')),
    alt.Y('percents', 
          axis=alt.Axis(format='%', title='Percents'),
          scale=alt.Scale(domain=(.03,.08))),
    color='rates',
    strokeDash='rates'
).properties(height=200, width=300,
            title='by Fiscal Year')
    
line_qtr = alt.Chart(qtrs).mark_line(point=True).encode(
    alt.X('time:O',
          axis=alt.Axis(title='Quarters')),
    alt.Y('percents', 
          axis=alt.Axis(title='', format='%'),
          scale=alt.Scale(domain=(.03,.08))),
    color='rates',
    strokeDash='rates'
).properties(height=200, width=200,
            title='in 2019 by Quarter')

line_mon = alt.Chart(mons).mark_bar().encode(
    alt.X('rates',
          axis=alt.Axis(title='Reserve Type')),
    alt.Y('percents', 
          axis=alt.Axis(title='',format='%'),
          scale=alt.Scale(domain=(.03,.08))),
    color='rates',
    strokeDash='rates'
).properties(height=200, width=200, title='in December 2019')

percents = alt.hconcat(line_yr, line_qtr, line_mon).properties(title='Precentages')


In [197]:
alt.vconcat(reserves, percents).properties(title='Loan Loss Reserves and NPLs')


alt.VConcatChart(...)